# DeepSurv for Remaining Process Runtime Prediction





In [1]:
# optional install

# import pip
# def install(package):
#     pip.main(["install",package])
# install("pm4py")

# install("missingno")
# import missingno as msno

# install("pycox")
# install("sklearn")
# install("torch")
# install("sklearn_pandas ")


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper

import torch
import torchtuples as tt

from pycox.datasets import metabric
from pycox.models import CoxPH
from pycox.evaluation import EvalSurv
import random


In [3]:
np.random.seed(1234)
_ = torch.manual_seed(1234)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Data preprocess for case attributes

Event attributes data is processed using Prefix length bucketing.

In [4]:
df = pd.read_csv('2017.csv')

In [5]:
df.head()

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID
0,Created,User_1,A_Create Application,Application,Application_652823628,complete,2016-01-01 09:51:15.304000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,statechange,User_1,A_Submitted,Application,ApplState_1582051990,complete,2016-01-01 09:51:15.352000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Created,User_1,W_Handle leads,Workflow,Workitem_1298499574,schedule,2016-01-01 09:51:15.774000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,2016-01-01 09:52:36.392000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,2016-01-01 09:52:36.403000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:

def preprocess_data(df):
    data = df.copy()

    # convert timestamp to datetime format
    data['time:timestamp'] = pd.to_datetime(data['time:timestamp'])
    data.sort_values(['case:concept:name', 'time:timestamp'], inplace=True)

    # calculate the whole process duration
    data['duration'] = data.groupby('case:concept:name')['time:timestamp'].transform(lambda x: x.max() - x.min())
    
    # create feature vector
    feature = data.drop_duplicates(subset=['case:concept:name'])

    # process case attributes
    feature.drop(columns=['Action','org:resource','concept:name','EventID','EventOrigin','lifecycle:transition','time:timestamp','OfferID', 
                          'FirstWithdrawalAmount','NumberOfTerms','Accepted','MonthlyCost','Selected','CreditScore','OfferedAmount'], inplace=True)
    
    feature['duration'] = feature['duration'].dt.total_seconds() / 86400.0
    
    # introduce event indicator
    event_indicator = data[data['concept:name'] == 'A_Pending']['case:concept:name'].unique()
    feature['event'] = feature['case:concept:name'].isin(event_indicator).astype(int)
    
    # one-hot coding 
    feature = pd.get_dummies(feature, columns=['case:LoanGoal', 'case:ApplicationType'])
    
     # save the feature from case attributes
    feature.to_csv('feature.csv', index=False)
    
    return feature



In [ ]:
def load_bucket_and_merge(t, feature):
    # load buckets to get event attributes 
    bucket = pd.read_csv(f'b/data{t}.csv')

    # merge with feature from case attributes
    bucket = pd.merge(bucket, feature, on='case:concept:name', how='left')

    return bucket


In [ ]:
def split_data(bucket):
    df_test = bucket.sample(frac=0.2, random_state=10)
    df_comp = df_test[df_test['event'] == 1]
    df_train = bucket.drop(df_test.index)
    df_val = df_train.sample(frac=0.2, random_state=11)
    df_train = df_train.drop(df_val.index)

    return df_train, df_val, df_test, df_comp


In [ ]:
def feature_transforms(df_train, df_val, df_test, df_comp):
    # standardize the numerical covariates, leave the binary variables as is
    cols_standardize = ['Duration','last_dur','Created','Deleted','Obtained','Released','statechange','Application',
                        'Offer','Workflow',
                       'case:RequestedAmount', 'FirstWithdrawalAmount','CreditScore', 'OfferedAmount',
                        'A_Complete','A_Concept','A_Create Application','A_Incomplete',
                        'A_Submitted','A_Validating','O_Accepted','O_Cancelled','O_Create Offer','O_Created',
                        'O_Returned','O_Sent (mail and online)','O_Sent (online only)',
                        'W_Call after offers','W_Call incomplete files',
                        'W_Complete application','W_Handle leads','W_Validate application']

    cols_leave = ['case:LoanGoal_Boat', 'case:LoanGoal_Business goal',
        'case:LoanGoal_Car', 'case:LoanGoal_Caravan / Camper',
        'case:LoanGoal_Debt restructuring',
        'case:LoanGoal_Existing loan takeover',
        'case:LoanGoal_Extra spending limit', 'case:LoanGoal_Home improvement',
        'case:LoanGoal_Motorcycle', 'case:LoanGoal_Not speficied',
        'case:LoanGoal_Other, see explanation','NumberOfTerms',
        'case:LoanGoal_Remaining debt home', 'case:LoanGoal_Tax payments',
        'case:LoanGoal_Unknown',  'Accepted','case:ApplicationType_Limit raise','case:ApplicationType_New credit',
            'Selected'] 

    standardize = [([col], StandardScaler()) for col in cols_standardize]
    leave = [(col, None) for col in cols_leave]
    x_mapper = DataFrameMapper(standardize + leave)

    # variables needs to be of type 'float32', as required by pytorch
    x_train = x_mapper.fit_transform(df_train).astype('float32')
    x_val = x_mapper.transform(df_val).astype('float32')
    x_test = x_mapper.transform(df_test).astype('float32')
    x_comp = x_mapper.transform(df_comp).astype('float32')
    
    return x_train, x_val, x_test, x_comp


## Deep learning-based survival analysis model

DeepSurv is primarily extends the Cox Proportional Hazards model, by incorporating a deep neural network to learn intricate patterns and relationships from the data. 

In [ ]:
def train_model(x_train, y_train, val):
    # Deep neural net
    in_features = x_train.shape[1]
    num_nodes = [128, 128, 128, 128]
    out_features = 1
    batch_norm = True
    dropout = 0.4 
    output_bias = False

    x_train, y_train = x_train.to(device), y_train.to(device)
    val = tuple(data.to(device) for data in val)

    net = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm, dropout, output_bias=output_bias)
    
    # Training the model, using the `Adam` optimizer
    model = CoxPH(net, tt.optim.Adam).to(device)
    batch_size = 128

    # Instead of choosing a learning rate, use the scheme proposed by [Smith 2017](https://arxiv.org/pdf/1506.01186.pdf) 
    # to find a suitable learning rate with `model.lr_finder`. 
    # See [this post](https://towardsdatascience.com/finding-good-learning-rate-and-the-one-cycle-policy-7159fe1db5d6) for an explanation.
    lrfinder = model.lr_finder(x_train, y_train, batch_size, tolerance=10)
    best_lr = lrfinder.get_best_lr()

    # We include the `EarlyStopping` callback to stop training when the validation loss stops improving. 
    # After training, this callback will also load the best performing model in terms of validation loss.
    callbacks = [tt.callbacks.EarlyStopping()]
    verbose = True
    log = model.fit(x_train, y_train, batch_size, 512, callbacks, verbose, val_data=val, val_batch_size=batch_size)
    
    return model


## Multiple predictor approach

Build one predictor for each bucket.


In [ ]:

mae_list = []
random_state = 42

feature = preprocess_data(df)

for t in range(1, 21):

    # load buckets
    bucket = load_bucket_and_merge(t, feature)

    # data split
    df_train, df_val, df_test, df_comp = split_data(bucket)

    # Feature transforms
    x_train, x_val, x_test, x_comp = feature_transforms(df_train, df_val, df_test, df_comp)

    get_target = lambda df: (df['duration'].values, df['event'].values)
    y_train = get_target(df_train)
    y_val = get_target(df_val)
    durations_train, events_train = get_target(df_train)
    durations_val, events_val = get_target(df_val)
    durations_test, events_test = get_target(df_test)
    durations_comp, events_comp = get_target(df_comp)
    val = x_val, y_val

    # Training the model
    model = train_model(x_train, y_train, val)

    # get the partial log-likelihood
    model.partial_log_likelihood(*val).mean()

    # get the non-parametric baseline hazard estimates
    _ = model.compute_baseline_hazards()

    # Prediction of DeepSurv, returning the survival estimates as a dataframe
    s_val = model.predict_surv_df(x_val)
    s_train = model.predict_surv_df(x_train)

    # determine the threshod by grid search
    thresholds = np.linspace(0.4, 1, num=100)
    best_val = None
    min_loss_v = float('inf')

    for threshold in thresholds:
        survival_v = s_val.apply(lambda col: (col <= threshold).idxmax())
        mae_v = np.mean(np.abs(survival_v - durations_val))

        if mae_v < min_loss_v:
            min_loss_v = mae_v
            best_val = threshold

    print("Best Threshold:", best_val)

    # Prediction on testset
    surv = model.predict_surv_df(x_comp)
    threshold = best_val
    survival_time = []
    survival_time = surv.apply(lambda col: (col <= threshold).idxmax())

    mae = np.mean(np.abs(survival_time - durations_comp))
    print("mae", mae)
    mae_list.append(mae)

print('averge MAE', np.mean(mae_list))
print('std', np.std(mae_list))
